In [1]:
from vllm import LLM, SamplingParams
import json

model = "/home2/xjw/LLaMA-Factory/models/llama3_lora_sft"
llm = LLM(model=model, tensor_parallel_size=1)

/home2/xjw/anaconda3/envs/nlp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-06-04 03:26:57,471	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


INFO 06-04 03:26:57 llm_engine.py:161] Initializing an LLM engine (v0.4.3) with config: model='/home2/xjw/LLaMA-Factory/models/llama3_lora_sft', speculative_config=None, tokenizer='/home2/xjw/LLaMA-Factory/models/llama3_lora_sft', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=/home2/xjw/LLaMA-Factory/models/llama3_lora_sft)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 06-04 03:27:06 model_runner.py:146] Loading model weights took 14.9595 GB
INFO 06-04 03:27:07 gpu_executor.py:83] # GPU blocks: 27895, # CPU blocks: 2048
INFO 06-04 03:27:09 model_runner.py:854] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 06-04 03:27:09 model_runner.py:858] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 06-04 03:27:13 model_runner.py:924] Graph capturing finished in 4 secs.


In [2]:
# 这里改了下，原来 "if result: " 的判断会出问题
def build_prompt(instruction, input, resp1, resp2, result=None, explain=None, ref=None):
    rsp = f"### Response 1:\n{resp1}\n\n### Response 2:\n{resp2}"

    if input:
        input_sequence = f"Below are two responses for a given task. The task is defined by the Instruction with an Input that provides further context. Evaluate the responses and generate a reference answer for the task.\n\n### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n{rsp}\n\n### Evaluation:\n"
    else:
        input_sequence = f"Below are two responses for a given task. The task is defined by the Instruction. Evaluate the responses and generate a reference answer for the task.\n\n### Instruction:\n{instruction}\n\n{rsp}\n\n### Evaluation:\n"


    output_sequence = f"{result}\n\n### Reason: {explain}\n\n### Reference: {ref}\n"
    
    return input_sequence, output_sequence

In [3]:
pandalm_test_set_path = "/home2/xjw/PandaLM/data/testset-v1.json"

pandalm_test_set = []
with open(pandalm_test_set_path, "r") as f:
  pandalm_test_set = json.load(f)

In [4]:
pandalm_test_set_prompt = []
for i in range(len(pandalm_test_set)):
  for j in range(3):
    pandalm_test_set_prompt.append(
      build_prompt(
        instruction=pandalm_test_set[i]['instruction'],
        input=pandalm_test_set[i]['input'] if 'input' in pandalm_test_set[i] else None,
        resp1=pandalm_test_set[i]['response1'],
        resp2=pandalm_test_set[i]['response2'],
        result=pandalm_test_set[i]['annotator' + str(j+1)] ,
      )
    )

In [5]:
prompt_template = """<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n{input}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"""

inputs = [ prompt_template.format(input = i[0]) for i in pandalm_test_set_prompt[ : : 3]]


In [6]:
print(inputs[0])
print(len(inputs))

<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Below are two responses for a given task. The task is defined by the Instruction with an Input that provides further context. Evaluate the responses and generate a reference answer for the task.

### Instruction:
The sentence you are given might be too wordy, complicated, or unclear. Rewrite the sentence and make your writing clearer by keeping it concise. Whenever possible, break complex sentences into multiple sentences and eliminate unnecessary words.

### Input:
If you have any questions about my rate or if you find it necessary to increase or decrease the scope for this project, please let me know.

### Response 1:
If you have any questions about my rate, please let me know.

### Response 2:
If you have any questions, please let me know.

### Evaluation:
<|eot_id|><|start_header_id|>assistant<|end_header_id|>


999


In [7]:
stop_tokens = ["<|eot_id|>", "<|end_of_text|>"]
sampling_params = SamplingParams(temperature=0.0, top_p=1.0, max_tokens=1024, stop=stop_tokens)

completions = llm.generate(inputs, sampling_params=sampling_params)

generated_responses = [output.outputs[0].text for output in completions]

with open("pandalm_test_set_output.json", "w") as f:
  json.dump(generated_responses, f)


Processed prompts: 100%|██████████| 999/999 [00:58<00:00, 17.18it/s, Generation Speed: 1730.42 toks/s]


In [6]:
import json
with open("pandalm_test_set_output.json", "r") as f:
  generated_responses = json.load(f)

In [7]:
print(generated_responses[2])

1

### Reason: Response 1 is better because it is more concise and still conveys the same meaning as the original sentence.

### Reference: If you have any questions about my rate, please let me know.



In [5]:
ans = [ res.split("\n\n")[0] for res in generated_responses]

ans = [ "0" if a == "Tie" else a for a in ans]
print(len(ans))
print(ans)

999
['1', '1', '1', '1', '2', '2', '1', '1', '1', '1', '1', '1', '1', '2', '2', '1', '1', '1', '2', '2', '2', '2', '2', '2', '2', '2', '2', '1', '1', '2', '2', '1', '2', '2', '2', '2', '2', '2', '2', '1', '2', '2', '1', '2', '2', '2', '2', '2', '1', '2', '2', '2', '1', '0', '1', '2', '1', '2', '1', '1', '0', '1', '1', '2', '1', '1', '1', '1', '2', '1', '2', '1', '2', '1', '0', '0', '0', '0', '2', '2', '2', '2', '2', '2', '1', '1', '2', '1', '1', '1', '1', '2', '2', '2', '2', '2', '1', '1', '2', '1', '2', '2', '1', '2', '1', '1', '2', '1', '1', '1', '2', '1', '2', '2', '2', '2', '1', '1', '1', '2', '2', '2', '2', '2', '2', '2', '1', '0', '0', '1', '1', '0', '0', '2', '2', '1', '1', '1', '1', '2', '0', '0', '0', '0', '0', '0', '0', '2', '1', '1', '1', '1', '1', '2', '1', '0', '2', '0', '1', '0', '0', '0', '1', '1', '0', '1', '1', '1', '2', '0', '1', '2', '1', '2', '2', '1', '2', '1', '2', '2', '1', '1', '2', '1', '2', '1', '1', '2', '2', '2', '2', '1', '2', '2', '1', '2', '2', '2', '1', 

In [11]:
correct = 0
category_1 = 0
pred_1 = 0
category_1_correct = 0
category_2 = 0
pred_2 = 0
category_2_correct = 0
category_3 = 0
pred_3 = 0
category_3_correct = 0

for j in range(len(pandalm_test_set)):
  for i in range(3):
    label = pandalm_test_set_prompt[j * 3 + i][1].split("\n\n")[0]
    if label == ans[j]:
        correct += 1
        if ans[j] == "0":
          category_1_correct += 1
        if ans[j] == "1":
          category_2_correct += 1
        if ans[j] == "2":
          category_3_correct += 1
    
    if ans[j] == "0":
      pred_1 += 1
    if ans[j] == "1":
      pred_2 += 1
    if ans[j] == "2":
      pred_3 += 1
      
    if label == "0":
      category_1 += 1
    if label == "1":
      category_2 += 1
    if label == "2":
      category_3 += 1   

print(f"Accuracy: {correct/len(pandalm_test_set_prompt) * 100:.2f}")

def f1_score(p, r):
  return 2 * p * r / (p + r)

p1 = category_1_correct / pred_1
r1 = category_1_correct / category_1
p2 = category_2_correct / pred_2
r2 = category_2_correct / category_2
p3 = category_3_correct / pred_3
r3 = category_3_correct / category_3

print(f"F1: {(f1_score(p1, r1) + f1_score(p2, r2) + f1_score(p3, r3) ) * 100 / 3 :.2f}")

Accuracy: 75.34
F1: 66.84


: 